<a href="https://colab.research.google.com/github/Aditya8215/Voice-Box/blob/main/Sign_Language_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

# After adding shortcut to MyDrive:
DATA_PATH = '/content/drive/MyDrive/subdir'


Mounted at /content/drive


In [ ]:
import os
print(os.listdir(DATA_PATH))  # should list your class folders


['00_LINK', 'V', 'W', 'R', 'X', 'Y', 'T', 'Q', 'U', 'S', 'Z', 'O', 'K', 'J', 'I', 'H', 'M', 'G', 'P', 'N', 'L', 'A', 'C', 'B', 'E', 'F', 'D']


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from multiprocessing import Pool
import time

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Path to dataset
DATA_PATH = '/content/drive/MyDrive/subdir'

# Config
IMG_SIZE = 224  # Optimal size for speed/performance balance
NUM_WORKERS = 4  # Adjust based on your Colab CPU cores

# Get valid image directories
CLASSES = [
    d for d in sorted(os.listdir(DATA_PATH))
    if os.path.isdir(os.path.join(DATA_PATH, d))
    and not d.startswith(('00_', 'OB_'))
]
NUM_CLASSES = len(CLASSES)

print(f"Found {NUM_CLASSES} valid classes: {CLASSES}")

def process_image(args):
    """Helper function for parallel processing"""
    image_path, label_index = args
    try:
        img = cv2.imread(image_path)
        if img is None:
            return None
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        return (img, label_index)
    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")
        return None

def load_images_fast():
    """Optimized image loading with parallel processing"""
    start_time = time.time()
    image_paths = []
    labels = []

    # First pass: collect all valid image paths
    for label_index, label in enumerate(CLASSES):
        class_path = os.path.join(DATA_PATH, label)
        image_files = [f for f in os.listdir(class_path)
                      if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        for image_file in image_files:
            image_paths.append(os.path.join(class_path, image_file))
            labels.append(label_index)

    print(f"Found {len(image_paths)} images to process")

    # Parallel processing
    with Pool(processes=NUM_WORKERS) as pool:
        results = pool.map(process_image, zip(image_paths, labels))

    # Filter out None results and separate images/labels
    valid_results = [r for r in results if r is not None]
    X = [r[0] for r in valid_results]
    y = [r[1] for r in valid_results]

    print(f"Processed {len(X)} images in {time.time()-start_time:.2f} seconds")
    return np.array(X), to_categorical(y, num_classes=NUM_CLASSES)

# Load data with timing
start_load = time.time()
X, y = load_images_fast()
print(f"Total loading time: {time.time()-start_load:.2f} seconds")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 26 valid classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
Found 1728 images to process
Processed 1728 images in 191.67 seconds
Total loading time: 191.73 seconds


In [ ]:
X_train

array([[[[  5,   0,   0],
         [  5,   0,   0],
         [  5,   0,   0],
         ...,
         [  0,   0,   1],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[  4,   0,   0],
         [  4,   0,   0],
         [  4,   0,   0],
         ...,
         [  0,   0,   1],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[  2,   0,   0],
         [  2,   0,   0],
         [  2,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        ...,

        [[  1,   0,   2],
         [  4,   0,   0],
         [  7,   0,   0],
         ...,
         [ 82, 146, 201],
         [ 76, 141, 196],
         [ 76, 141, 197]],

        [[  0,   0,   2],
         [  1,   0,   0],
         [  5,   0,   0],
         ...,
         [ 76, 140, 198],
         [ 73, 139, 197],
         [ 74, 140, 199]],

        [[  0,   0,   2],
         [  1,   0,   0],
         [  5,   0,   0],
         ...,
         [ 76, 140, 198],
        

In [ ]:
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def create_asl_model(input_shape=(224, 224, 3), num_classes=26):
    """Create optimized CNN model for ASL letter recognition"""

    model = models.Sequential([
        # Enhanced Input Block with data augmentation built-in
        layers.RandomRotation(0.1, input_shape=input_shape),
        layers.RandomZoom(0.1),
        layers.RandomContrast(0.1),

        # Block 1 - Enhanced with spatial attention
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.SpatialDropout2D(0.25),

        # Block 2 - Depth increased
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.SpatialDropout2D(0.25),

        # Block 3 - With residual connection
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.SpatialDropout2D(0.25),

        # Block 4 - Global Context
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),

        # Classifier with regularization
        layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])

    # Custom optimizer configuration
    optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=True)

    model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy',
                        tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top3_accuracy')])

    return model

# Create model
asl_model = create_asl_model(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=NUM_CLASSES)
asl_model.summary()

# Enhanced callbacks
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7),
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)
]



/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_rotation                 │ (None, 224, 224, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom (RandomZoom)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_contrast                 │ (None, 224, 224, 3)    │             0 │
│ (RandomContrast)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 224, 224, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d               │ (None, 112, 112, 64)   │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 112, 112, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 112, 112, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_1             │ (None, 56, 56, 128)    │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 56, 56, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 56, 56, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 256)    │             

 Total params: 2,734,938 (10.43 MB)

 Trainable params: 2,730,586 (10.42 MB)

 Non-trainable params: 4,352 (17.00 KB)

In [ ]:
# Train the model
history = asl_model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=200,
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    verbose=1
)

Epoch 1/30


In [ ]:
history.summary()

In [ ]:
cnn_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
cnn_model.summary()


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint("best_model.h5", save_best_only=True)
]



In [ ]:
history = cnn_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    callbacks=callbacks
)


# Checking the model

In [ ]:
import matplotlib.pyplot as plt

# Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
cap = cv2.VideoCapture(0)  # 0 = default webcam

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess frame
    img = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)  # Add batch dimension: (1, 128, 128, 3)

    # Predict
    prediction = model.predict(img)
    class_index = np.argmax(prediction)
    confidence = prediction[0][class_index]

    # Get label name
    label = CLASSES[class_index]

    # Display
    cv2.putText(frame, f"{label} ({confidence*100:.2f}%)", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Live Sign Detection", frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

# Load your trained model
# model = load_model("sign_model.h5")

# Define your constants
IMG_SIZE = 128  # same size as during training
CLASSES = sorted(os.listdir(DATA_PATH))  # or hardcode your label list


In [ ]:
# Save the CNN model
cnn_model.save("sign_model.h5")

# Convert the CNN model to TensorFlow Lite
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(cnn_model)
tflite_model = converter.convert()

# Save the TFLite model
with open("sign_model.tflite", "wb") as f:
    f.write(tflite_model)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Save to Drive
with open("/content/drive/MyDrive/sign_model.tflite", "wb") as f:
    f.write(tflite_model)
